In [20]:

import numpy as np
import pandas as pd
import seaborn as sb
sb.set_style("dark")
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [21]:
# In Kaggle uncomment below line
# train_df_org = pd.read_csv('kaggle/input/digit-recognizer/train.csv')

# In Kaggle comment-out below line
train_df_org = pd.read_csv('kaggle/input/digit-recognizer/train.csv',  nrows=100)
# test_df_org = np.genfromtxt('kaggle/input/digit-recognizer/test.csv', delimiter=',', skip_header=1)
# sample_submission_df = pd.read_csv('kaggle/input/digit-recognizer/sample_submission.csv')

In [22]:
# train_df_org.shape, test_df_org.shape, sample_submission_df.shape
train_df_org.shape

(100, 785)

In below the labels variable contains the labels and then from the 'train_df_org' (which contains all the actual image pixel) I will drop 'label' column

In [ ]:
labels = train_df_org['label']
labels.shape


In [24]:
train_df_org.drop(['label'], axis=1, inplace=True)
train_df_org.shape


(100, 784)

In [25]:
# Data-preprocessing: Standardizing the train dataframe
# (x - mean) / s.d

standardized_data = StandardScaler().fit_transform(train_df_org)
print(standardized_data.shape)

(100, 784)


#### Step-B => Build the Covariance-Matrix S which is $(X^T * X)$



In [ ]:
# Now build the c-variance matrix which is : A^T * A
sample_data = standardized_data

# Matrix multiplication with numpy
covariance_matrix = np.matmul(sample_data.T, sample_data)

# As the sample_data has 784 columns, so the co-variance matrix shape
# should be 784 * 784
print('Shape of Co-variance matrix = ', covariance_matrix.shape)



#### Step-C => Get the EigneVectors of this Covariance-Matrix S for projecting onto a 2-D space. These Eigenvectors will be sorted by the value of λ.

In [ ]:
from scipy.linalg import eigh

# subset_by_index is an optional and iterable parameter
# If provided, this two-element iterable defines the start and the end indices
# of the desired eigenvalues (ascending order and 0-indexed).
# To return only the second smallest to fifth smallest eigenvalues, [1, 4] is used.
# [n-3, n-1] returns the largest three.
# Only available with “evr”, “evx”, and “gvx” drivers. The entries are directly converted to integers via int().
eigenvalues, eigenvectors = eigh(covariance_matrix, subset_by_index=(782, 783) )
print('Shape of Eigenvectors ', eigenvectors.shape )
